### Manipulation of Peaks de novo results of ETNP 2017 P2 samples LC-MS/MS data using python.

Starting with:

    Peaks de novo results (.csv) of PTM-optimized database searches > ALC 50%

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

We don't have technical duplicates here, sadly, unlike the MED4 Pro samples. I exported PeaksDN search results CSVs into my ETNP 2017 git repo:

In [1]:
cd /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt


In [2]:
ls

231/  233/  243/  273/  278/  378/


In [3]:
ls 273

ETNP_SKQ17_DENOVO_170_273-965m-trap_DN50_15ppm.csv
ETNP_SKQ17_DENOVO_170_273-965m-trap_DN50_15ppm.xml


In [4]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

## 273: 965 m +P filtered on 0.3 um GF-75

In [5]:
# read the CSV into a dataframe using the pandas read_csv function
peaks273_50 = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP_SKQ17_DENOVO_170_273-965m-trap_DN50_15ppm.csv")

# also make a dataframe but keep only peptides >80% ALC
peaks273_80 = peaks273_50.loc[peaks273_50['ALC (%)'] >= 80].copy()

# how many de novo sequence candidates >50% ALC?
print("# redundant Peaks peptides >50% ALC in dataframe", len(peaks273_50))
# how many de novo sequence candidates >80ALC?
print("# redundant Peaks peptides >80% ALC in dataframe", len(peaks273_80))

#look at the dataframe
peaks273_50.head()

# redundant Peaks peptides >50% ALC in dataframe 8563
# redundant Peaks peptides >80% ALC in dataframe 1039


,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,Area,Mass,ppm,PTM,local confidence (%),tag (>=0%),mode
0,6,12978,20181003_4-19_273_965m_top.raw,LLYEEEK,7,98,7,462.2459,2,40.82,2450000.0,922.4647,13.6,NaN,98 99 97 99 99 99 99,LLYEEEK,CID
1,6,16792,20181003_4-19_273_965m_top.raw,FEDTNLAALHAK,12,97,12,665.3448,2,50.59,643000.0,1328.6724,2.0,NaN,95 100 99 99 98 99 99 97 99 97 97 95,FEDTNLAALHAK,CID
2,6,27770,20181003_4-19_273_965m_top.raw,FLELLK,6,97,6,381.7418,2,72.75,293000.0,761.4687,0.4,NaN,94 97 100 97 99 98,FLELLK,CID
3,6,26188,20181003_4-19_273_965m_top.raw,LLEWLK,6,97,6,401.2476,2,69.91,927000.0,800.4796,1.3,NaN,98 98 98 95 98 96,LLEWLK,CID
4,6,6251,20181003_4-19_273_965m_top.raw,EEDLER,6,97,6,395.6828,2,22.37,7190000.0,789.3504,0.9,NaN,99 100 97 98 99 91,EEDLER,CID


In [6]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks273_50['A'] = peaks273_50['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks273_50['C'] = peaks273_50['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks273_50['D'] = peaks273_50['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks273_50['E'] = peaks273_50['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks273_50['F'] = peaks273_50['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks273_50['G'] = peaks273_50['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks273_50['H'] = peaks273_50['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks273_50 output, there will be no isoleucines (they're lumped in with leucines)
peaks273_50['I'] = peaks273_50['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks273_50['K'] = peaks273_50['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks273_50['L'] = peaks273_50['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks273_50['M'] = peaks273_50['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks273_50['N'] = peaks273_50['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks273_50['P'] = peaks273_50['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks273_50['Q'] = peaks273_50['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks273_50['R'] = peaks273_50['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks273_50['S'] = peaks273_50['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks273_50['T'] = peaks273_50['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks273_50['V'] = peaks273_50['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks273_50['W'] = peaks273_50['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks273_50['Y'] = peaks273_50['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks273_50['c-carb'] = peaks273_50['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks273_50['m-oxid'] = peaks273_50['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks273_50['n-deam'] = peaks273_50['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks273_50['q-deam'] = peaks273_50['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks273_50['k-hydr'] = peaks273_50['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
# no more P hydroxylation in final searches
# peaks273_50['p-hydr'] = peaks273_50['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks273_50['r-meth'] = peaks273_50['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks273_50['stripped peptide'] = peaks273_50['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks273_50['stripped length'] = peaks273_50['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks273_50['ptm-total'] = peaks273_50['c-carb'] + peaks273_50['m-oxid'] + peaks273_50['n-deam'] \
+ peaks273_50['q-deam'] + peaks273_50['k-hydr'] + peaks273_50['r-meth']

# calculate NAAF numerator for each peptide k
peaks273_50['NAAF num.'] = peaks273_50['Area'] / peaks273_50['stripped length']

# write modified dataframe to new csv file
peaks273_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_15ppm_DN50.csv")


# check out the results
peaks273_50.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,c-carb,m-oxid,n-deam,q-deam,k-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.
0,6,12978,20181003_4-19_273_965m_top.raw,LLYEEEK,7,98,7,462.2459,2,40.82,...,0,0,0,0,0,0,LLYEEEK,7,0,3.500000e+05
1,6,16792,20181003_4-19_273_965m_top.raw,FEDTNLAALHAK,12,97,12,665.3448,2,50.59,...,0,0,0,0,0,0,FEDTNLAALHAK,12,0,5.358333e+04
2,6,27770,20181003_4-19_273_965m_top.raw,FLELLK,6,97,6,381.7418,2,72.75,...,0,0,0,0,0,0,FLELLK,6,0,4.883333e+04
3,6,26188,20181003_4-19_273_965m_top.raw,LLEWLK,6,97,6,401.2476,2,69.91,...,0,0,0,0,0,0,LLEWLK,6,0,1.545000e+05
4,6,6251,20181003_4-19_273_965m_top.raw,EEDLER,6,97,6,395.6828,2,22.37,...,0,0,0,0,0,0,EEDLER,6,0,1.198333e+06


In [7]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks273_50['A'].sum(),
        'C': peaks273_50['C'].sum(),
        'D': peaks273_50['D'].sum(),
        'E': peaks273_50['E'].sum(),
        'F': peaks273_50['F'].sum(),
        'G': peaks273_50['G'].sum(),
        'H': peaks273_50['H'].sum(),
        'I': peaks273_50['I'].sum(),
        'K': peaks273_50['K'].sum(),
        'L': peaks273_50['L'].sum(),
        'M': peaks273_50['M'].sum(),
        'N': peaks273_50['N'].sum(),
        'P': peaks273_50['P'].sum(),
        'Q': peaks273_50['Q'].sum(),
        'R': peaks273_50['R'].sum(),
        'S': peaks273_50['S'].sum(),
        'T': peaks273_50['T'].sum(),
        'V': peaks273_50['V'].sum(),
        'W': peaks273_50['W'].sum(),
        'Y': peaks273_50['Y'].sum(),
        'c-carb': peaks273_50['c-carb'].sum(),
        'm-oxid': peaks273_50['m-oxid'].sum(),
        'n-deam': peaks273_50['n-deam'].sum(),
        'q-deam': peaks273_50['q-deam'].sum(),
        'k-hydr': peaks273_50['k-hydr'].sum(),
        'r-meth': peaks273_50['r-meth'].sum(),
        'Total area': peaks273_50['Area'].sum(),
        'Total length': peaks273_50['stripped length'].sum()
       }

totalpeaks273_50 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',\
                                               'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', \
                                               'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'r-meth', \
                                               'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks273_50['% C w/ carb'] = totalpeaks273_50['c-carb'] / totalpeaks273_50['C'] 

# calculate percentage of M's that are oxidized
totalpeaks273_50['% M w/ oxid'] = totalpeaks273_50['m-oxid'] / totalpeaks273_50['M'] 

# calculate percentage of N's that are deamidated
totalpeaks273_50['% N w/ deam'] = totalpeaks273_50['n-deam'] / totalpeaks273_50['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks273_50['% Q w/ deam'] = totalpeaks273_50['q-deam'] / totalpeaks273_50['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks273_50['% K w/ hydr'] = totalpeaks273_50['k-hydr'] / totalpeaks273_50['K'] 

# calculate percentage of P's that are hydroxylated
#totalpeaks273_50['% P w/ hydr'] = totalpeaks273_50['p-hydr'] / totalpeaks273_50['K'] 

# calculate percentage of R's that are methylated
totalpeaks273_50['% R w/ meth'] = totalpeaks273_50['r-meth'] / totalpeaks273_50['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks273_50['NAAF denom.'] = totalpeaks273_50['Total area'] / totalpeaks273_50['Total length']

# write modified dataframe to new txt file
totalpeaks273_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_15ppm_DN50_totals.csv")

totalpeaks273_50.head()

,A,C,D,E,F,G,H,I,K,L,...,r-meth,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% R w/ meth,NAAF denom.
sample total,4422,1356,3513,4649,2699,2842,2191,0,8317,7629,...,2380,1.499713e+10,65916,1.0,0.38811,0.221365,0.026044,0.192978,0.465753,227518.835159


In [8]:
# use the calculated NAAF factor (in totalpeaks273 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF50 = 227518.835159

# use NAAF >50% ALC to get NAAF
peaks273_50['NAAF factor'] = (peaks273_50['NAAF num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaks273_AA50 = peaks273_50[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                             'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', \
                             'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'r-meth']].copy()

# multiply the NAAF50 factor by the AAs to normalize its abundance by peak area and peptide length

peaks273_AA50['A-NAAF50'] = peaks273_AA50['A'] * peaks273_50['NAAF factor']
peaks273_AA50['C-NAAF50'] = peaks273_AA50['C'] * peaks273_50['NAAF factor']
peaks273_AA50['D-NAAF50'] = peaks273_AA50['D'] * peaks273_50['NAAF factor']
peaks273_AA50['E-NAAF50'] = peaks273_AA50['E'] * peaks273_50['NAAF factor']
peaks273_AA50['F-NAAF50'] = peaks273_AA50['F'] * peaks273_50['NAAF factor']
peaks273_AA50['G-NAAF50'] = peaks273_AA50['G'] * peaks273_50['NAAF factor']
peaks273_AA50['H-NAAF50'] = peaks273_AA50['H'] * peaks273_50['NAAF factor']
peaks273_AA50['I-NAAF50'] = peaks273_AA50['I'] * peaks273_50['NAAF factor']
peaks273_AA50['K-NAAF50'] = peaks273_AA50['K'] * peaks273_50['NAAF factor']
peaks273_AA50['L-NAAF50'] = peaks273_AA50['L'] * peaks273_50['NAAF factor']
peaks273_AA50['M-NAAF50'] = peaks273_AA50['M'] * peaks273_50['NAAF factor']
peaks273_AA50['N-NAAF50'] = peaks273_AA50['N'] * peaks273_50['NAAF factor']
peaks273_AA50['P-NAAF50'] = peaks273_AA50['P'] * peaks273_50['NAAF factor']
peaks273_AA50['Q-NAAF50'] = peaks273_AA50['Q'] * peaks273_50['NAAF factor']
peaks273_AA50['R-NAAF50'] = peaks273_AA50['R'] * peaks273_50['NAAF factor']
peaks273_AA50['S-NAAF50'] = peaks273_AA50['S'] * peaks273_50['NAAF factor']
peaks273_AA50['T-NAAF50'] = peaks273_AA50['T'] * peaks273_50['NAAF factor']
peaks273_AA50['V-NAAF50'] = peaks273_AA50['V'] * peaks273_50['NAAF factor']
peaks273_AA50['W-NAAF50'] = peaks273_AA50['W'] * peaks273_50['NAAF factor']
peaks273_AA50['Y-NAAF50'] = peaks273_AA50['Y'] * peaks273_50['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaks273_AA50['ccarb-NAAF50'] = peaks273_AA50['c-carb'] * peaks273_AA50['NAAF factor']
peaks273_AA50['moxid-NAAF50'] = peaks273_AA50['m-oxid'] * peaks273_AA50['NAAF factor']
peaks273_AA50['ndeam-NAAF50'] = peaks273_AA50['n-deam'] * peaks273_AA50['NAAF factor']
peaks273_AA50['qdeam-NAAF50'] = peaks273_AA50['q-deam'] * peaks273_AA50['NAAF factor']
peaks273_AA50['khydr-NAAF50'] = peaks273_AA50['k-hydr'] * peaks273_AA50['NAAF factor']
peaks273_AA50['rmeth-NAAF50'] = peaks273_AA50['r-meth'] * peaks273_AA50['NAAF factor']

# write the dataframe to a new csv
peaks273_AA50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_15ppm_DN50_NAAF.csv")

peaks273_AA50.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,I,...,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50,khydr-NAAF50,rmeth-NAAF50
0,LLYEEEK,1.538334,0,0,0,3,0,0,0,0,...,0.000000,0.0,0.000000,1.538334,0.0,0.0,0.0,0.0,0.0,0.0
1,FEDTNLAALHAK,0.235512,3,0,1,1,1,0,1,0,...,0.235512,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,FLELLK,0.214634,0,0,0,1,1,0,0,0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,LLEWLK,0.679065,0,0,0,1,0,0,0,0,...,0.000000,0.0,0.679065,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,EEDLER,5.266963,0,0,1,3,0,0,0,0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# made a new dataframe that contains the sums of NAAF normalized AAs for comet273 PeaksDN results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaks273_AA50['NAAF factor'].sum(),
        'A-NAAF': peaks273_AA50['A-NAAF50'].sum(),
        'C-NAAF': peaks273_AA50['C-NAAF50'].sum(),
        'D-NAAF': peaks273_AA50['D-NAAF50'].sum(),
        'E-NAAF': peaks273_AA50['E-NAAF50'].sum(),
        'F-NAAF': peaks273_AA50['F-NAAF50'].sum(),
        'G-NAAF': peaks273_AA50['G-NAAF50'].sum(),
        'H-NAAF': peaks273_AA50['H-NAAF50'].sum(),
        'I-NAAF': peaks273_AA50['I-NAAF50'].sum(),
        'K-NAAF': peaks273_AA50['K-NAAF50'].sum(),
        'L-NAAF': peaks273_AA50['L-NAAF50'].sum(),
        'M-NAAF': peaks273_AA50['M-NAAF50'].sum(),
        'N-NAAF': peaks273_AA50['N-NAAF50'].sum(),
        'P-NAAF': peaks273_AA50['P-NAAF50'].sum(),
        'Q-NAAF': peaks273_AA50['Q-NAAF50'].sum(),
        'R-NAAF': peaks273_AA50['R-NAAF50'].sum(),
        'S-NAAF': peaks273_AA50['S-NAAF50'].sum(),
        'T-NAAF': peaks273_AA50['T-NAAF50'].sum(),
        'V-NAAF': peaks273_AA50['V-NAAF50'].sum(),
        'W-NAAF': peaks273_AA50['W-NAAF50'].sum(),
        'Y-NAAF': peaks273_AA50['Y-NAAF50'].sum(),
        'C-carb-NAAF': peaks273_AA50['ccarb-NAAF50'].sum(),
        'M-oxid-NAAF': peaks273_AA50['moxid-NAAF50'].sum(),
        'N-deam-NAAF': peaks273_AA50['ndeam-NAAF50'].sum(),
        'Q-deam-NAAF': peaks273_AA50['qdeam-NAAF50'].sum(),
        'K-hydr-NAAF': peaks273_AA50['khydr-NAAF50'].sum(),
        'R-meth-NAAF': peaks273_AA50['rmeth-NAAF50'].sum()
       }

totalpeaks273_AA50 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF', 'K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF', 'K-hydr-NAAF',\
                                                   'R-meth-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, P, and Rs


totalpeaks273_AA50['% C w/ carb. NAAF'] = totalpeaks273_AA50['C-carb-NAAF'] / totalpeaks273_AA50['C-NAAF']
totalpeaks273_AA50['% M w/ oxid. NAAF'] = totalpeaks273_AA50['M-oxid-NAAF'] / totalpeaks273_AA50['M-NAAF']
totalpeaks273_AA50['% N w/ deam. NAAF'] = totalpeaks273_AA50['N-deam-NAAF'] / totalpeaks273_AA50['N-NAAF']
totalpeaks273_AA50['% Q w/ deam. NAAF'] = totalpeaks273_AA50['Q-deam-NAAF'] / totalpeaks273_AA50['Q-NAAF']
totalpeaks273_AA50['% K w/ hydr. NAAF'] = totalpeaks273_AA50['K-hydr-NAAF'] / totalpeaks273_AA50['K-NAAF']
totalpeaks273_AA50['% R w/ meth. NAAF'] = totalpeaks273_AA50['R-meth-NAAF'] / totalpeaks273_AA50['R-NAAF']

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks273_AA50['NAAF check'] = totalpeaks273_AA50['NAAF'] / 227518.835159

# write modified dataframe to new txt file, same name + totals
totalpeaks273_AA50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_DN50_NAAF_totals.csv")

totalpeaks273_AA50.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,Q-deam-NAAF,K-hydr-NAAF,R-meth-NAAF,% C w/ carb. NAAF,% M w/ oxid. NAAF,% N w/ deam. NAAF,% Q w/ deam. NAAF,% K w/ hydr. NAAF,% R w/ meth. NAAF,NAAF check
sample total,10470.513465,7972.133459,1101.944016,3277.06471,4559.730698,2260.65203,3302.775941,1997.843113,0.0,7636.358816,...,166.558401,2217.433796,4094.63867,1.0,0.518918,0.312881,0.066905,0.290378,0.520858,0.04602


### Same process but for de novo peptide >80 % ALC:

In [10]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks273_80['A'] = peaks273_80['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks273_80['C'] = peaks273_80['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks273_80['D'] = peaks273_80['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks273_80['E'] = peaks273_80['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks273_80['F'] = peaks273_80['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks273_80['G'] = peaks273_80['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks273_80['H'] = peaks273_80['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks273_80 output, there will be no isoleucines (they're lumped in with leucines)
peaks273_80['I'] = peaks273_80['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks273_80['K'] = peaks273_80['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks273_80['L'] = peaks273_80['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks273_80['M'] = peaks273_80['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks273_80['N'] = peaks273_80['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks273_80['P'] = peaks273_80['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks273_80['Q'] = peaks273_80['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks273_80['R'] = peaks273_80['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks273_80['S'] = peaks273_80['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks273_80['T'] = peaks273_80['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks273_80['V'] = peaks273_80['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks273_80['W'] = peaks273_80['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks273_80['Y'] = peaks273_80['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks273_80['c-carb'] = peaks273_80['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks273_80['m-oxid'] = peaks273_80['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks273_80['n-deam'] = peaks273_80['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks273_80['q-deam'] = peaks273_80['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks273_80['k-hydr'] = peaks273_80['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
#peaks273_80['p-hydr'] = peaks273_80['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks273_80['r-meth'] = peaks273_80['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks273_80['stripped peptide'] = peaks273_80['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks273_80['stripped length'] = peaks273_80['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks273_80['ptm-total'] = peaks273_80['c-carb'] + peaks273_80['m-oxid'] + peaks273_80['n-deam'] + \
peaks273_80['q-deam'] + peaks273_80['k-hydr'] + peaks273_80['r-meth']

# calculate NAAF numerator for each peptide k
peaks273_80['NAAF num.'] = peaks273_80['Area'] / peaks273_80['stripped length']

# write modified dataframe to new csv file
peaks273_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_15ppm_DN80.csv")

# check out the results
peaks273_80.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,c-carb,m-oxid,n-deam,q-deam,k-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.
0,6,12978,20181003_4-19_273_965m_top.raw,LLYEEEK,7,98,7,462.2459,2,40.82,...,0,0,0,0,0,0,LLYEEEK,7,0,3.500000e+05
1,6,16792,20181003_4-19_273_965m_top.raw,FEDTNLAALHAK,12,97,12,665.3448,2,50.59,...,0,0,0,0,0,0,FEDTNLAALHAK,12,0,5.358333e+04
2,6,27770,20181003_4-19_273_965m_top.raw,FLELLK,6,97,6,381.7418,2,72.75,...,0,0,0,0,0,0,FLELLK,6,0,4.883333e+04
3,6,26188,20181003_4-19_273_965m_top.raw,LLEWLK,6,97,6,401.2476,2,69.91,...,0,0,0,0,0,0,LLEWLK,6,0,1.545000e+05
4,6,6251,20181003_4-19_273_965m_top.raw,EEDLER,6,97,6,395.6828,2,22.37,...,0,0,0,0,0,0,EEDLER,6,0,1.198333e+06


In [11]:
# made a new dataframe that contains the sums of certain columns in modified 
#peptide dataframe above (for >80% ALC)

index = ['sample total']

data = {'A': peaks273_80['A'].sum(),
        'C': peaks273_80['C'].sum(),
        'D': peaks273_80['D'].sum(),
        'E': peaks273_80['E'].sum(),
        'F': peaks273_80['F'].sum(),
        'G': peaks273_80['G'].sum(),
        'H': peaks273_80['H'].sum(),
        'I': peaks273_80['I'].sum(),
        'K': peaks273_80['K'].sum(),
        'L': peaks273_80['L'].sum(),
        'M': peaks273_80['M'].sum(),
        'N': peaks273_80['N'].sum(),
        'P': peaks273_80['P'].sum(),
        'Q': peaks273_80['Q'].sum(),
        'R': peaks273_80['R'].sum(),
        'S': peaks273_80['S'].sum(),
        'T': peaks273_80['T'].sum(),
        'V': peaks273_80['V'].sum(),
        'W': peaks273_80['W'].sum(),
        'Y': peaks273_80['Y'].sum(),
        'c-carb': peaks273_80['c-carb'].sum(),
        'm-oxid': peaks273_80['m-oxid'].sum(),
        'n-deam': peaks273_80['n-deam'].sum(),
        'q-deam': peaks273_80['q-deam'].sum(),
        'k-hydr': peaks273_80['k-hydr'].sum(),
        'r-meth': peaks273_80['r-meth'].sum(),
        'Total area': peaks273_80['Area'].sum(),
        'Total length': peaks273_80['stripped length'].sum()
       }

totalpeaks273_80 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M',\
                                               'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb',\
                                               'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'r-meth',\
                                               'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks273_80['% C w/ carb'] = totalpeaks273_80['c-carb'] / totalpeaks273_80['C'] 

# calculate percentage of M's that are oxidized
totalpeaks273_80['% M w/ oxid'] = totalpeaks273_80['m-oxid'] / totalpeaks273_80['M'] 

# calculate percentage of N's that are deamidated
totalpeaks273_80['% N w/ deam'] = totalpeaks273_80['n-deam'] / totalpeaks273_80['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks273_80['% Q w/ deam'] = totalpeaks273_80['q-deam'] / totalpeaks273_80['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks273_80['% K w/ hydr'] = totalpeaks273_80['k-hydr'] / totalpeaks273_80['K'] 

# calculate percentage of P's that are hydroxylated
#totalpeaks273_80['% P w/ hydr'] = totalpeaks273_80['p-hydr'] / totalpeaks273_80['K'] 

# calculate percentage of R's that are methylated
totalpeaks273_80['% R w/ meth'] = totalpeaks273_80['r-meth'] / totalpeaks273_80['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks273_80['NAAF denom.'] = totalpeaks273_80['Total area'] / totalpeaks273_80['Total length']

# write modified dataframe to new txt file
totalpeaks273_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_15ppm_DN80_totals.csv")

totalpeaks273_80.head()

,A,C,D,E,F,G,H,I,K,L,...,r-meth,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% R w/ meth,NAAF denom.
sample total,608,92,484,794,330,325,112,0,785,1200,...,140,4.573608e+09,7947,1.0,0.539095,0.343696,0.092742,0.105732,0.260223,575513.781427


In [12]:
# use the calculated NAAF factor (in totalpeaks273 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF80 = 575513.781427

# use NAAF >80% ALC to get NAAF
peaks273_80['NAAF factor'] = (peaks273_80['NAAF num.'])/NAAF80

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaks273_AA80 = peaks273_80[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                             'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                             'n-deam', 'q-deam', 'k-hydr', 'r-meth']].copy()

# multiply the NAAF80 factor by the AAs to normalize its abundance by peak area and peptide length

peaks273_AA80['A-NAAF80'] = peaks273_AA80['A'] * peaks273_80['NAAF factor']
peaks273_AA80['C-NAAF80'] = peaks273_AA80['C'] * peaks273_80['NAAF factor']
peaks273_AA80['D-NAAF80'] = peaks273_AA80['D'] * peaks273_80['NAAF factor']
peaks273_AA80['E-NAAF80'] = peaks273_AA80['E'] * peaks273_80['NAAF factor']
peaks273_AA80['F-NAAF80'] = peaks273_AA80['F'] * peaks273_80['NAAF factor']
peaks273_AA80['G-NAAF80'] = peaks273_AA80['G'] * peaks273_80['NAAF factor']
peaks273_AA80['H-NAAF80'] = peaks273_AA80['H'] * peaks273_80['NAAF factor']
peaks273_AA80['I-NAAF80'] = peaks273_AA80['I'] * peaks273_80['NAAF factor']
peaks273_AA80['K-NAAF80'] = peaks273_AA80['K'] * peaks273_80['NAAF factor']
peaks273_AA80['L-NAAF80'] = peaks273_AA80['L'] * peaks273_80['NAAF factor']
peaks273_AA80['M-NAAF80'] = peaks273_AA80['M'] * peaks273_80['NAAF factor']
peaks273_AA80['N-NAAF80'] = peaks273_AA80['N'] * peaks273_80['NAAF factor']
peaks273_AA80['P-NAAF80'] = peaks273_AA80['P'] * peaks273_80['NAAF factor']
peaks273_AA80['Q-NAAF80'] = peaks273_AA80['Q'] * peaks273_80['NAAF factor']
peaks273_AA80['R-NAAF80'] = peaks273_AA80['R'] * peaks273_80['NAAF factor']
peaks273_AA80['S-NAAF80'] = peaks273_AA80['S'] * peaks273_80['NAAF factor']
peaks273_AA80['T-NAAF80'] = peaks273_AA80['T'] * peaks273_80['NAAF factor']
peaks273_AA80['V-NAAF80'] = peaks273_AA80['V'] * peaks273_80['NAAF factor']
peaks273_AA80['W-NAAF80'] = peaks273_AA80['W'] * peaks273_80['NAAF factor']
peaks273_AA80['Y-NAAF80'] = peaks273_AA80['Y'] * peaks273_80['NAAF factor']

# multiply the NAAF80 factor by the PTMs normalize its abundance by peak area and peptide length

peaks273_AA80['ccarb-NAAF80'] = peaks273_AA80['c-carb'] * peaks273_AA80['NAAF factor']
peaks273_AA80['moxid-NAAF80'] = peaks273_AA80['m-oxid'] * peaks273_AA80['NAAF factor']
peaks273_AA80['ndeam-NAAF80'] = peaks273_AA80['n-deam'] * peaks273_AA80['NAAF factor']
peaks273_AA80['qdeam-NAAF80'] = peaks273_AA80['q-deam'] * peaks273_AA80['NAAF factor']
peaks273_AA80['khydr-NAAF80'] = peaks273_AA80['k-hydr'] * peaks273_AA80['NAAF factor']
peaks273_AA80['rmeth-NAAF80'] = peaks273_AA80['r-meth'] * peaks273_AA80['NAAF factor']

# write the dataframe to a new csv
peaks273_AA80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_15ppm_DN80_NAAF.csv")

peaks273_AA80.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,I,...,T-NAAF80,V-NAAF80,W-NAAF80,Y-NAAF80,ccarb-NAAF80,moxid-NAAF80,ndeam-NAAF80,qdeam-NAAF80,khydr-NAAF80,rmeth-NAAF80
0,LLYEEEK,0.608152,0,0,0,3,0,0,0,0,...,0.000000,0.0,0.000000,0.608152,0.0,0.0,0.0,0.0,0.0,0.0
1,FEDTNLAALHAK,0.093105,3,0,1,1,1,0,1,0,...,0.093105,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,FLELLK,0.084852,0,0,0,1,1,0,0,0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,LLEWLK,0.268456,0,0,0,1,0,0,0,0,...,0.000000,0.0,0.268456,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,EEDLER,2.082197,0,0,1,3,0,0,0,0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# made a new dataframe that contains the sums of NAAF normalized AAs for comet273 PeaksDN results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaks273_AA80['NAAF factor'].sum(),
        'A-NAAF': peaks273_AA80['A-NAAF80'].sum(),
        'C-NAAF': peaks273_AA80['C-NAAF80'].sum(),
        'D-NAAF': peaks273_AA80['D-NAAF80'].sum(),
        'E-NAAF': peaks273_AA80['E-NAAF80'].sum(),
        'F-NAAF': peaks273_AA80['F-NAAF80'].sum(),
        'G-NAAF': peaks273_AA80['G-NAAF80'].sum(),
        'H-NAAF': peaks273_AA80['H-NAAF80'].sum(),
        'I-NAAF': peaks273_AA80['I-NAAF80'].sum(),
        'K-NAAF': peaks273_AA80['K-NAAF80'].sum(),
        'L-NAAF': peaks273_AA80['L-NAAF80'].sum(),
        'M-NAAF': peaks273_AA80['M-NAAF80'].sum(),
        'N-NAAF': peaks273_AA80['N-NAAF80'].sum(),
        'P-NAAF': peaks273_AA80['P-NAAF80'].sum(),
        'Q-NAAF': peaks273_AA80['Q-NAAF80'].sum(),
        'R-NAAF': peaks273_AA80['R-NAAF80'].sum(),
        'S-NAAF': peaks273_AA80['S-NAAF80'].sum(),
        'T-NAAF': peaks273_AA80['T-NAAF80'].sum(),
        'V-NAAF': peaks273_AA80['V-NAAF80'].sum(),
        'W-NAAF': peaks273_AA80['W-NAAF80'].sum(),
        'Y-NAAF': peaks273_AA80['Y-NAAF80'].sum(),
        'C-carb-NAAF': peaks273_AA80['ccarb-NAAF80'].sum(),
        'M-oxid-NAAF': peaks273_AA80['moxid-NAAF80'].sum(),
        'N-deam-NAAF': peaks273_AA80['ndeam-NAAF80'].sum(),
        'Q-deam-NAAF': peaks273_AA80['qdeam-NAAF80'].sum(),
        'K-hydr-NAAF': peaks273_AA80['khydr-NAAF80'].sum(),
        'R-meth-NAAF': peaks273_AA80['rmeth-NAAF80'].sum()
       }

totalpeaks273_AA80 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF', 'K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF', 'K-hydr-NAAF',\
                                                   'R-meth-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, P, and Rs


totalpeaks273_AA80['% C w/ carb. NAAF'] = totalpeaks273_AA80['C-carb-NAAF'] / totalpeaks273_AA80['C-NAAF']
totalpeaks273_AA80['% M w/ oxid. NAAF'] = totalpeaks273_AA80['M-oxid-NAAF'] / totalpeaks273_AA80['M-NAAF']
totalpeaks273_AA80['% N w/ deam. NAAF'] = totalpeaks273_AA80['N-deam-NAAF'] / totalpeaks273_AA80['N-NAAF']
totalpeaks273_AA80['% Q w/ deam. NAAF'] = totalpeaks273_AA80['Q-deam-NAAF'] / totalpeaks273_AA80['Q-NAAF']
totalpeaks273_AA80['% K w/ hydr. NAAF'] = totalpeaks273_AA80['K-hydr-NAAF'] / totalpeaks273_AA80['K-NAAF']
totalpeaks273_AA80['% R w/ meth. NAAF'] = totalpeaks273_AA80['R-meth-NAAF'] / totalpeaks273_AA80['R-NAAF']

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks273_AA80['NAAF check'] = totalpeaks273_AA80['NAAF'] / 575513.781427

# write modified dataframe to new txt file, same name + totals
totalpeaks273_AA80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_15ppm_DN80_NAAF_totals.csv")

totalpeaks273_AA80.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,Q-deam-NAAF,K-hydr-NAAF,R-meth-NAAF,% C w/ carb. NAAF,% M w/ oxid. NAAF,% N w/ deam. NAAF,% Q w/ deam. NAAF,% K w/ hydr. NAAF,% R w/ meth. NAAF,NAAF check
sample total,1036.224507,1118.497458,82.725479,373.446897,652.536012,194.830328,415.549484,50.396599,0.0,425.180004,...,48.608954,64.717628,300.322334,1.0,0.765826,0.4006,0.204383,0.152212,0.369559,0.001801


### Visualizing the results

In [14]:
print("ALC max: ", peaks273_80['ALC (%)'].max())
print("ALC min: ", peaks273_80['ALC (%)'].min())

ALC max:  98
ALC min:  80


### Exporting txt files of stripped peptides at confidence cutoffs:

In [15]:
# keep only peptide column >50% ALC
pep273_50_moddup = peaks273_50[["Peptide"]]

# keep only the stripped peptide column for the >50% ALC
# this is what we'll use for UniPept input, etc
pep273_50_dup = peaks273_50[["stripped peptide"]]

# deduplicate both of these lists
pep273_50_mod = pep273_50_moddup.drop_duplicates()
pep273_50 = pep273_50_dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified peptides in 273:", len(pep273_50_moddup))
print("Deduplicated modified peptides in 273:", len(pep273_50_mod))
print("Total strippled peptides in 273:", len(pep273_50_dup))
print("Deduplicated stripped peptides in 273:", len(pep273_50))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep273_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_15ppm_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 

!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_15ppm_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_15ppm_DN50_stripped_peptides.fas


Total modified peptides in 273: 8563
Deduplicated modified peptides in 273: 8490
Total strippled peptides in 273: 8563
Deduplicated stripped peptides in 273: 8009


In [16]:
# keep only peptide column >80% ALC
pep273_80_moddup = peaks273_80[["Peptide"]]

# keep only the stripped peptide column for the >80% ALC
# this is what we'll use for UniPept input, etc
pep273_80_dup = peaks273_80[["stripped peptide"]]

# deduplicate both of these lists
pep273_80_mod = pep273_80_moddup.drop_duplicates()
pep273_80 = pep273_80_dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified peptides in 273:", len(pep273_80_moddup))
print("Deduplicated modified peptides in 273:", len(pep273_80_mod))
print("Total strippled peptides in 273:", len(pep273_80_dup))
print("Deduplicated stripped peptides in 273:", len(pep273_80))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep273_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_15ppm_DN80_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 

!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_15ppm_DN80_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/273/ETNP-SKQ17-273-965m-trap_PTMopt_15ppm_DN80_stripped_peptides.fas


Total modified peptides in 273: 1039
Deduplicated modified peptides in 273: 1025
Total strippled peptides in 273: 1039
Deduplicated stripped peptides in 273: 1011
